In [7]:
import os
os.chdir('/home/evgenii/work/itmo/DroneRescue')
import numpy as np
from agent.llm_agent import LLMAgent
from environment.environment import Environment
from environment.constants import OBSTACLE_MAP, OBJECT_MAP, COLOR_MAP

In [8]:
AREA_SIZE = (100, 100)
NUM_TARGETS = 3
NUM_OBSTACLES = 50
OBSERVATION_AREA = (9,9)

import os
api_key = os.getenv("OPENAI_API_KEY")
agent = LLMAgent(area_size=AREA_SIZE, object_map=OBJECT_MAP, observation_area=OBSERVATION_AREA, api_key=api_key, chat_gpt_model="gpt-4o-2024-08-06")

In [9]:
agent_position = (0,0)
env = Environment(obstacles_map=OBSTACLE_MAP,
                color_map=COLOR_MAP,
                object_map=OBJECT_MAP,
                area_size=AREA_SIZE,
                num_targets=NUM_TARGETS,
                num_obstacles=NUM_OBSTACLES
                )
observation_area_of_agent = agent.observation_area
new_position, done, observation, obs_binary_mask, metadata = env.step(4, agent_position, observation_area_of_agent, 1)
agent_position = new_position
env.render_env('./exp_1/environment.jpg')

# Update visited map
agent.update_visited_map(observation, obs_binary_mask)

# Add current agent position to the visited map
visited_map_wiht_cut_agent_position = agent.visited_map
visited_map_wiht_cut_agent_position[agent_position] = 4 

# Add current agent position to the local observation
observation_with_cur_agent_position = observation.copy()
observation_with_cur_agent_position[metadata["agent_position_in_local_observation"]] = 4 


for step in range(10):
    # print(visited_map_wiht_cut_agent_position)
    response = agent.generate_action_by_model(visited_map_wiht_cut_agent_position, observation_with_cur_agent_position)
    action = response["action"]
    explaination = response["explaination"]
    speed = response["speed"]
    print(f"Step_{step}: A: {action} S: {speed}. Explainataion: {explaination}")

    new_position, done, observation, obs_binary_mask, metadata = env.step(action, agent_position, observation_area_of_agent, speed)
    agent_position = new_position

    # Update visited map
    agent.update_visited_map(observation, obs_binary_mask)

    # Add current agent position to the visited map
    visited_map_wiht_cut_agent_position = agent.visited_map.copy()
    visited_map_wiht_cut_agent_position[agent_position] = 4 

    # Add current agent position to the local observation
    observation_with_cur_agent_position = observation.copy()
    observation_with_cur_agent_position[metadata["agent_position_in_local_observation"]] = 4 

    env.render_visited_map(visited_map_wiht_cut_agent_position, f'./exp_1/visited_map_{step}.jpg')
    env.render_observation(observation_with_cur_agent_position, f'./exp_1/observation_{step}.jpg')

Step_0: A: 3 S: 5. Explainataion: The current observation shows a clear area with no obstacles or target points immediately around the drone. The drone is at position 4 and is surrounded by background (0) and unvisited areas (3). To efficiently explore the unvisited areas, it's optimal to move in a direction with a higher speed. Moving right seems clear and will explore more unvisited areas. Hence, moving right with a speed of 5 should efficiently cover new ground without hitting obstacles.
Step_1: A: 3 S: 5. Explainataion: The drone is on an open path with no immediate obstacles in the observation area. The best approach now is to explore and cover more ground efficiently. Moving to the right is a viable option since the path is clear, and it allows us to explore unvisited areas.
Step_2: A: 3 S: 5. Explainataion: The drone is surrounded by open areas in all directions, so it's optimal to explore further. Moving right will allow us to quickly scout unexplored areas and potentially find